## Встраивания

В предыдущем примере мы работали с высокоразмерными векторами мешка слов длиной `vocab_size` и явно преобразовывали низкоразмерные векторы позиционного представления в разреженное одноразрядное представление. Такое одноразрядное представление неэффективно с точки зрения памяти, а также каждое слово рассматривается независимо от других, то есть одноразрядные векторы не выражают никакого семантического сходства между словами.

В этом разделе мы продолжим изучать набор данных **News AG**. Для начала давайте загрузим данные и возьмем некоторые определения из предыдущего ноутбука.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Что такое эмбеддинг?

Идея **эмбеддинга** заключается в представлении слов в виде плотных векторов меньшей размерности, которые каким-то образом отражают семантическое значение слова. Позже мы обсудим, как создавать осмысленные эмбеддинги слов, но пока просто будем рассматривать эмбеддинги как способ уменьшения размерности вектора слова.

Таким образом, слой эмбеддинга принимает слово на вход и выдает выходной вектор заданного размера `embedding_size`. В некотором смысле, это похоже на слой `Linear`, но вместо того, чтобы принимать вектор в формате one-hot, он может принимать номер слова в качестве входных данных.

Используя слой эмбеддинга в качестве первого слоя нашей сети, мы можем перейти от модели bag-of-words к модели **embedding bag**, где мы сначала преобразуем каждое слово в тексте в соответствующий эмбеддинг, а затем вычисляем некоторую агрегирующую функцию для всех этих эмбеддингов, например, `sum`, `average` или `max`.

![Изображение, показывающее классификатор с использованием эмбеддинга для пяти слов последовательности.](../../../../../translated_images/ru/embedding-classifier-example.b77f021a7ee67eee.webp)

Наша нейронная сеть-классификатор начнется со слоя эмбеддинга, затем слоя агрегации и линейного классификатора поверх него:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Работа с переменной длиной последовательностей

Из-за особенностей этой архитектуры, минибатчи для нашей сети нужно будет формировать определённым образом. В предыдущем разделе, при использовании мешка слов (bag-of-words), все тензоры BoW в минибатче имели одинаковый размер `vocab_size`, независимо от фактической длины текстовой последовательности. Однако при переходе к словарным эмбеддингам количество слов в каждом текстовом образце будет варьироваться, и при объединении этих образцов в минибатчи нам потребуется добавлять некоторую "заполняющую" информацию (padding).

Это можно сделать с помощью той же техники, предоставляя функцию `collate_fn` источнику данных:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Обучение классификатора эмбеддингов

Теперь, когда мы определили подходящий загрузчик данных, мы можем обучить модель, используя функцию обучения, которую мы определили в предыдущем разделе:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Примечание**: Здесь мы обучаемся только на 25 тысячах записей (меньше, чем один полный эпох), чтобы сэкономить время, но вы можете продолжить обучение, написать функцию для обучения на нескольких эпохах и поэкспериментировать с параметром скорости обучения, чтобы достичь более высокой точности. Вы должны быть в состоянии достичь точности около 90%.


### Слой EmbeddingBag и представление последовательностей переменной длины

В предыдущей архитектуре нам приходилось дополнять все последовательности до одинаковой длины, чтобы они подходили для минибатча. Это не самый эффективный способ представления последовательностей переменной длины — другой подход заключается в использовании **вектора смещений**, который содержит смещения всех последовательностей, хранящихся в одном большом векторе.

![Изображение, показывающее представление последовательности со смещением](../../../../../translated_images/ru/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: На изображении выше показана последовательность символов, но в нашем примере мы работаем с последовательностями слов. Однако общий принцип представления последовательностей с помощью вектора смещений остается тем же.

Для работы с представлением через смещения мы используем слой [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Он похож на `Embedding`, но принимает вектор содержимого и вектор смещений в качестве входных данных, а также включает слой усреднения, который может быть `mean`, `sum` или `max`.

Вот модифицированная сеть, использующая `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Чтобы подготовить набор данных для обучения, нам нужно предоставить функцию преобразования, которая подготовит вектор смещения:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Обратите внимание, что, в отличие от всех предыдущих примеров, наша сеть теперь принимает два параметра: вектор данных и вектор смещения, которые имеют разные размеры. Аналогично, наш загрузчик данных также предоставляет нам 3 значения вместо 2: как текстовые, так и векторы смещения предоставляются в качестве признаков. Поэтому нам нужно немного скорректировать нашу функцию обучения, чтобы учесть это:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Семантические эмбеддинги: Word2Vec

В нашем предыдущем примере слой эмбеддинга модели научился отображать слова в векторное представление, однако это представление не имело большого семантического значения. Было бы здорово научиться создавать такое векторное представление, при котором похожие слова или синонимы соответствовали бы векторами, близкими друг к другу по некоторой метрике расстояния (например, евклидовому расстоянию).

Для этого необходимо предварительно обучить модель эмбеддинга на большом наборе текстов определённым образом. Один из первых методов обучения семантических эмбеддингов называется [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Он основан на двух основных архитектурах, которые используются для создания распределённого представления слов:

 - **Непрерывный мешок слов** (Continuous bag-of-words, CBoW) — в этой архитектуре модель обучается предсказывать слово по окружающему контексту. Для данного n-грамма $(W_{-2},W_{-1},W_0,W_1,W_2)$ цель модели — предсказать $W_0$ на основе $(W_{-2},W_{-1},W_1,W_2)$.
 - **Непрерывный skip-gram** — противоположность CBoW. Модель использует окружающее окно контекстных слов для предсказания текущего слова.

CBoW работает быстрее, тогда как skip-gram медленнее, но лучше справляется с представлением редких слов.

![Изображение, показывающее алгоритмы CBoW и Skip-Gram для преобразования слов в векторы.](../../../../../translated_images/ru/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Чтобы поэкспериментировать с эмбеддингами Word2Vec, предварительно обученными на наборе данных Google News, мы можем использовать библиотеку **gensim**. Ниже приведён пример поиска слов, наиболее похожих на 'neural'.

> **Note:** При первом создании векторных представлений слов их загрузка может занять некоторое время!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Мы также можем вычислить векторные представления из слова, чтобы использовать их при обучении модели классификации (мы показываем только первые 20 компонентов вектора для ясности):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Замечательная особенность семантических встраиваний заключается в том, что можно изменять векторное кодирование, чтобы изменить семантику. Например, мы можем попросить найти слово, чье векторное представление будет максимально близко к словам *король* и *женщина*, и максимально далеко от слова *мужчина*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Оба метода, CBoW и Skip-Grams, являются "предсказательными" векторами, так как они учитывают только локальные контексты. Word2Vec не использует преимущества глобального контекста.

**FastText** расширяет возможности Word2Vec, обучая векторные представления для каждого слова и n-грамм символов, найденных внутри слова. Значения этих представлений затем усредняются в один вектор на каждом этапе обучения. Хотя это добавляет значительное количество вычислений на этапе предварительного обучения, оно позволяет векторным представлениям слов кодировать информацию о частях слова.

Другой метод, **GloVe**, использует идею матрицы совместной встречаемости и применяет нейронные методы для разложения матрицы совместной встречаемости на более выразительные и нелинейные векторные представления слов.

Вы можете поэкспериментировать, изменяя векторы на FastText и GloVe, так как gensim поддерживает несколько различных моделей векторных представлений слов.


## Использование предварительно обученных эмбеддингов в PyTorch

Мы можем изменить приведенный выше пример, чтобы заранее заполнить матрицу в нашем слое эмбеддингов семантическими эмбеддингами, такими как Word2Vec. Нужно учитывать, что словари предварительно обученных эмбеддингов и нашего текстового корпуса, скорее всего, не будут совпадать, поэтому мы инициализируем веса для отсутствующих слов случайными значениями:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Теперь давайте обучим нашу модель. Обратите внимание, что время, необходимое для обучения модели, значительно больше, чем в предыдущем примере, из-за большего размера слоя встраивания, а значит, и гораздо большего количества параметров. Также из-за этого нам может понадобиться обучить нашу модель на большем количестве примеров, если мы хотим избежать переобучения.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

В нашем случае мы не наблюдаем значительного увеличения точности, что, вероятно, связано с довольно различными словарями.  
Чтобы преодолеть проблему различных словарей, мы можем использовать одно из следующих решений:  
* Переобучить модель word2vec на нашем словаре  
* Загрузить наш набор данных с использованием словаря из предварительно обученной модели word2vec. Словарь, используемый для загрузки набора данных, можно указать во время загрузки.  

Последний подход кажется проще, особенно потому, что фреймворк PyTorch `torchtext` содержит встроенную поддержку для эмбеддингов. Например, мы можем создать словарь на основе GloVe следующим образом:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Загруженный словарь имеет следующие базовые операции:
* Словарь `vocab.stoi` позволяет преобразовать слово в его индекс в словаре.
* `vocab.itos` выполняет обратное действие — преобразует число в слово.
* `vocab.vectors` — это массив векторов эмбеддингов, поэтому, чтобы получить эмбеддинг слова `s`, нужно использовать `vocab.vectors[vocab.stoi[s]]`.

Вот пример манипуляции с эмбеддингами, чтобы продемонстрировать уравнение **kind-man+woman = queen** (мне пришлось немного подправить коэффициент, чтобы это сработало):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Чтобы обучить классификатор с использованием этих эмбеддингов, сначала нужно закодировать наш набор данных с помощью словаря GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Как мы видели выше, все векторные эмбеддинги хранятся в матрице `vocab.vectors`. Это делает загрузку этих весов в веса слоя эмбеддинга очень простой благодаря простому копированию:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Одна из причин, по которой мы не наблюдаем значительного увеличения точности, заключается в том, что некоторые слова из нашего набора данных отсутствуют в предварительно обученной словарной базе GloVe, и поэтому они фактически игнорируются. Чтобы преодолеть это, мы можем обучить собственные эмбеддинги на нашем наборе данных.


## Контекстуальные эмбеддинги

Одним из ключевых ограничений традиционных предварительно обученных представлений эмбеддингов, таких как Word2Vec, является проблема неоднозначности значений слов. Хотя предварительно обученные эмбеддинги могут захватывать часть смысла слов в контексте, каждое возможное значение слова кодируется в одном и том же эмбеддинге. Это может вызывать проблемы в последующих моделях, так как многие слова, такие как слово "play", имеют разные значения в зависимости от контекста, в котором они используются.

Например, слово "play" в этих двух предложениях имеет совершенно разные значения:
- Я ходил на **пьесу** в театр.
- Джон хочет **играть** со своими друзьями.

Предварительно обученные эмбеддинги, упомянутые выше, представляют оба этих значения слова "play" в одном и том же эмбеддинге. Чтобы преодолеть это ограничение, необходимо создавать эмбеддинги на основе **языковой модели**, которая обучена на большом корпусе текста и *знает*, как слова могут сочетаться в различных контекстах. Обсуждение контекстуальных эмбеддингов выходит за рамки данного урока, но мы вернемся к ним, когда будем говорить о языковых моделях в следующем разделе.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
